In [ ]:
# Установка необходимых библиотек
!pip install transformers datasets torch fsspec==2023.10.0

In [ ]:
# Импорт библиотек
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [ ]:
# Определяем устройство (CPU или GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка токенизатора и модели
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Перемещаем модель на устройство
model = model.to(device)

# Загрузка и подготовка данных
data = load_dataset("wikitext", "wikitext-2-raw-v1")

# Добавление токена для заполнения
tokenizer.pad_token = tokenizer.eos_token

# Подготовка данных
def preprocess_function(examples):
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    inputs["labels"] = inputs["input_ids"].copy()  # Добавление меток
    return inputs

tokenized_datasets = data.map(preprocess_function, batched=True, remove_columns=["text"])

# Подготовка данных для тренировки
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))  # Используем небольшую выборку
eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(500)) 

# Настройки тренировки
training_args = TrainingArguments(
    output_dir="./gpt_results",
    report_to="none",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    num_train_epochs=2,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
)

# Создание объекта Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Запуск обучения (Тренируем по необходимости)
# trainer.train()

# Генерация текста после обучения
input_text = "Artificial Intelligence is"
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Декодирование текста
print(tokenizer.decode(output[0], skip_special_tokens=True))


In [ ]:
# Укажите путь к checkpoint
model_name = "./gpt_results/checkpoint-500"

# Загрузка токенизатора и модели из checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Перемещение модели на устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Генерация текста с загруженной моделью
input_text = "Artificial Intelligence is"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)  # Перемещаем данные на устройство
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Декодирование текста
print(tokenizer.decode(output[0], skip_special_tokens=True))